In [1]:
import modelskill as ms
import xarray as xr
import pandas as pd
import numpy as np

(The following data structure can be automatically generated using a `NetworkMapper` from `mikeio1d`)

In [2]:
ds = xr.load_dataset("../tests/testdata/network.nc")
ds

<xarray.Dataset> Size: 231kB
Dimensions:     (time: 110, node: 259)
Coordinates:
  * time        (time) datetime64[ns] 880B 1994-08-07T16:35:00 ... 1994-08-07...
  * node        (node) int64 2kB 0 1 2 3 4 5 6 7 ... 252 253 254 255 256 257 258
Data variables:
    Discharge   (time, node) float32 114kB nan nan 5.72e-06 ... nan 0.01692 0.0
    WaterLevel  (time, node) float32 114kB 195.4 194.7 nan ... 188.5 nan nan

# Step 1: Create Network Model Result

Now we'll create a `NetworkModelResult` from the network dataset.

In [3]:
network_model = ms.NetworkModelResult(ds, item="WaterLevel")
network_model

<NetworkModelResult>: WaterLevel

# Step 2: Extract Model Data at Observation Nodes

Now we'll extract the network model data at each observation location.

In [4]:
def dummy_series(t):
    return 1.0 + 0.5 * np.sin(np.arange(t) * 0.1) + 0.1 * np.random.randn(t)

model_time = ds.time
n_sensors = 3
# Create DataFrame with observations for multiple nodes
obs_data = {}
for i in range(n_sensors):   
    obs_data[f'station_{i}'] = dummy_series(len(model_time))
    
obs_df = pd.DataFrame(obs_data, index=model_time)
obs_df.head()

,station_0,station_1,station_2
1994-08-07 16:35:00.000,0.923208,0.931876,1.015190
1994-08-07 16:36:01.870,1.071448,0.967669,1.119526
1994-08-07 16:37:07.560,0.932132,1.006483,1.215846
1994-08-07 16:38:55.828,0.957551,1.233982,1.106535
1994-08-07 16:39:55.828,1.240895,1.131755,1.123500


In [16]:
network_model.data.node.values

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 18

In [10]:
single_obs = ms.NodeObservation(obs_df, node=3, item=1)
ms.match(single_obs, network_model).skill()

,n,bias,rmse,urmse,mae,cc,si,r2
observation,,,,,,,,
station_1,110,194.974696,194.975334,0.498746,194.974696,-0.350483,0.474966,-303574.912093


In [9]:
multi_obs = ms.MultiNodeObservation(obs_df, nodes=[1, 3, 5])
ms.match(multi_obs, network_model).skill()

,n,bias,rmse,urmse,mae,cc,si,r2
observation,,,,,,,,
station_0,110,193.892757,193.894293,0.771780,193.892757,-0.419851,0.735063,-280544.095582
station_1,110,194.974696,194.975334,0.498746,194.974696,-0.350483,0.474966,-303574.912093
station_2,110,193.082056,193.084009,0.868361,193.082056,-0.266680,0.825668,-308413.962548
